In [3]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re

In [4]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [5]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [6]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [7]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [8]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [9]:
df['loc']

1     (1300 Monad Terrace, Miami Beach, FL 33139, US...
2     (8955 Collins Ave, Surfside, FL 33154, USA, (2...
3     (10201 Collins Ave, Bal Harbour, FL 33154, USA...
4     (6800 Fisher Island Dr, Miami Beach, FL 33109,...
5     (1300 Monad Terrace, Miami Beach, FL 33139, US...
6     (18555 Collins Ave, Sunny Isles Beach, FL 3316...
7     (1300 Monad Terrace, Miami Beach, FL 33139, US...
8     (18501 Collins Ave, Sunny Isles Beach, FL 3316...
9     (18501 Collins Ave, Sunny Isles Beach, FL 3316...
10    (1300 Monad Terrace, Miami Beach, FL 33139, US...
Name: loc, dtype: object

In [10]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [11]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Buyer Agent,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Monad Terrace\r1300 Monad Terr PHA\rMiami Beach,12/6/2021,12/6/2021,1,"$11,800,000.00",4863,"$2,426.49",Comp Sale,Dina Goldentayer,Monad Terrace 1300 Monad Terr PHA Miami Beach,Monad Terrace,1300 Monad Terr PHA Miami Beach,"(1300 Monad Terrace, Miami Beach, FL 33139, US...","(25.7840774, -80.1425416, 0.0)",25.784077,-80.142542,0.0
2,Arte Surfside\r8955 Collins Ave 502\rSurfside,12/6/2021,12/6/2021,1,"$10,975,000.00",3931,"$2,791.91",Comp Sale,Dina Goldentayer,Arte Surfside 8955 Collins Ave 502 Surfside,Arte Surfside,8955 Collins Ave 502 Surfside,"(8955 Collins Ave, Surfside, FL 33154, USA, (2...","(25.8758096, -80.12120449999999, 0.0)",25.875810,-80.121204,0.0
3,Oceana Bal Harbour Condo\r10203 Collins Ave 12...,2/13/2020,12/6/2021,630,"$8,940,000.00",3992,"$2,239.48",Roza Pereira,Sarah Gozlan,Oceana Bal Harbour Condo 10203 Collins Ave 120...,Oceana Bal Harbour Condo,10203 Collins Ave 1201 Bal Harbour,"(10201 Collins Ave, Bal Harbour, FL 33154, USA...","(25.8951514, -80.1230855, 0.0)",25.895151,-80.123086,0.0
4,Fisher Island\r6800 Fisher Island Dr 6842\rFis...,10/29/2019,12/7/2021,547,"$8,500,000.00",3620,"$2,348.07",Dora Puig,Katrina Campins PA,Fisher Island 6800 Fisher Island Dr 6842 Fishe...,Fisher Island,6800 Fisher Island Dr 6842 Fisher Island,"(6800 Fisher Island Dr, Miami Beach, FL 33109,...","(25.7623668, -80.14735929999999, 0.0)",25.762367,-80.147359,0.0
5,Monad Terrace\r1300 Monad Terr 5A\rMiami Beach,3/11/2021,12/6/2021,180,"$5,950,000.00",4308,"$2,253.79",Tracy Galya,Ann Nortmann PA,Monad Terrace 1300 Monad Terr 5A Miami Beach,Monad Terrace,1300 Monad Terr 5A Miami Beach,"(1300 Monad Terrace, Miami Beach, FL 33139, US...","(25.7840774, -80.1425416, 0.0)",25.784077,-80.142542,0.0
6,Porsche Design Tower\r18555 Collins Ave 1601\r...,12/6/2021,12/6/2021,1,"$5,750,000.00",3171,"$1,813.31",Comp Sale,Scott Shuffield,Porsche Design Tower 18555 Collins Ave 1601 Su...,Porsche Design Tower,18555 Collins Ave 1601 Sunny Isles Beach,"(18555 Collins Ave, Sunny Isles Beach, FL 3316...","(25.9482145, -80.1197839, 0.0)",25.948214,-80.119784,0.0
7,Monad Terrace\r1300 Monad Terr 6F\rMiami Beach,3/23/2021,12/6/2021,209,"$5,620,000.00",4679,"$2,114.37",Christian Kawas,Ronit Perez,Monad Terrace 1300 Monad Terr 6F Miami Beach,Monad Terrace,1300 Monad Terr 6F Miami Beach,"(1300 Monad Terrace, Miami Beach, FL 33139, US...","(25.7840774, -80.1425416, 0.0)",25.784077,-80.142542,0.0
8,Turnberry Ocean Club\r18501 Collins Ave 1504\r...,11/25/2021,12/8/2021,11,"$5,362,500.00",3252,"$1,648.99",Mariana Shulga,Ryan Mendell,Turnberry Ocean Club 18501 Collins Ave 1504 Su...,Turnberry Ocean Club,18501 Collins Ave 1504 Sunny Isles Beach,"(18501 Collins Ave, Sunny Isles Beach, FL 3316...","(25.9477873, -80.12022019999999, 0.0)",25.947787,-80.120220,0.0
9,Turnberry Ocean Club\r18501 Collins Ave 1903\r...,8/1/2021,12/7/2021,93,"$4,550,000.00",3760,"$1,463.02",Sylvia Fragos,Jonessa Guarino,Turnberry Ocean Club 18501 Collins Ave 1903 Su...,Turnberry Ocean Club,18501 Collins Ave 1903 Sunny Isles Beach,"(18501 Collins Ave, Sunny Isles Beach, FL 3316...","(25.9477873, -80.12022019999999, 0.0)",25.947787,-80.120220,0.0
10,Monad Terrace\r1300 Monad Terr 12B\rMiami Beach,12/6/2021,12/6/2021,1,"$4,000,000.00",2691,"$1,486.44",Comp Sale,Dina Goldentayer,Monad Terrace 1300 Monad Terr 12B Miami Beach,Monad Terrace,1300 Monad Terr 12B Miami Beach,"(1300 Monad Terrace, Miami Beach, FL 33139, US...","(25.7840774, -80.1425416, 0.0)",25.784077,-80.142542,0.0


Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [12]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Buyer Agent',
       'geo_address', 'building_name', 'address_only', 'loc', 'point', 'lat',
       'lon', 'altitude'],
      dtype='object')

In [13]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('December 5th - December 11th')


marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['Sq. Ft.'], r['address_only'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}${info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
m

In [14]:
m.save('index.html')